<a href="https://colab.research.google.com/github/jarinari/Algorithmic-Trading-Plugins/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

web_url = 'https://finviz.com/quote.ashx?t='

news_tables = {}
tickers = ['MSFT', 'AAPL', 'AMZN', 'NVDA', 'META', 'GOOGL', 'MA', 'AMD', 'GOOG', 'V']

for tick in tickers:
  url = web_url + tick
  req = Request(url=url,headers={"User-Agent": "Mozilla"})
  response = urlopen(req)
  html = BeautifulSoup(response,"html.parser")
  news_table = html.find(id='news-table')
  news_tables[tick] = news_table

news_list = []

for file_name, news_table in news_tables.items():
  for i in news_table.findAll('tr'):
    link = i.a
    if link is not None:
      text = link.get_text()
      date_scrape = i.td.text.split()
      if len(date_scrape) == 1:
        time = date_scrape[0]
      else:
        date = date_scrape[0]
        time = date_scrape[1]
        tick = file_name.split('_')[0]
        news_list.append([tick, date, time, text])

vader = SentimentIntensityAnalyzer()
columns = ['ticker', 'date', 'time', 'headline']
news_df = pd.DataFrame(news_list, columns=columns)
scores = news_df['headline'].apply(vader.polarity_scores).tolist()
scores_df = pd.DataFrame(scores)
news_df = news_df.join(scores_df, rsuffix='_right')

df_sentiment = news_df.groupby('ticker').agg({'compound': 'mean'}).sort_values('compound', ascending=False)
df_sentiment

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,compound
ticker,
NVDA,0.282233
AMD,0.205113
GOOG,0.204367
GOOGL,0.204367
MA,0.121000
V,0.069622
AMZN,0.067900
META,0.064292
MSFT,0.005943
